# 1. Введение

Как известно, специалист в области Data Science занимается построением моделей, которые помогают в принятии верных решений, — например, построением нейронной сети, которая по медицинским данным о пациенте способна предсказать развитие у него онкологии, или модели, которая будет прогнозировать цены акций на бирже.

Однако не все знают, что перед тем как построить свою модель, дата-сайентист проходит через огромный этап под названием Предобработка данных и их подготовка к анализу и подаче в модель.

Под предобработкой понимаются следующие этапы работы с данными:

очистка данных от аномальных значений (выбросов);
работа с пропущенными значениями;
удаление признаков, которые не несут полезной информации;
создание новых признаков;
преобразование признаков и приведение данных к необходимому для анализа и модели формату.
Этап подготовки данных является самым трудоёмким и времязатратным при работе с любой бизнес-задачей. В среднем он занимает около 60-70% общей работы специалиста!

?
Почему этот этап так важен?

Ответ на этот вопрос кроется в распространённой среди специалистов поговорке «Мусор на входе — мусор на выходе», что означает следующее: если данные плохо подготовлены, то и результат прогнозирования даже самой мощной в мире нейронной сети будет сильно разниться с действительностью.

## FEATURE ENGINEERING

Одним из этапов подготовки данных является удаление, преобразование и создание столбцов таблицы.
?
Зачем нужны такие манипуляции? 

Оказывается, что при правильных преобразованиях таблицы можно добиваться лучшего качества прогноза, а также извлекать новую информацию из данных и интерпретировать её для заказчика. 

Такой подход часто называют Feature Engineering, или генерацией признаков (фичей).

→ На самом деле, в этом термине заложен более глубокий смысл, ведь Feature Engineering — это целая методология получения более качественных и более производительных моделей за счёт манипуляций над данными. Специалисты часто называют данную методологию настоящим искусством, которое может быть освоено лишь с годами практики решения задач, ведь необходимо быть экспертом в исследуемой предметной области, чтобы понимать, как признаки влияют друг на друга и какое преобразование стоит к ним применить. 

В данном модуле мы сделаем первые шаги в изучении темы подготовки данных и рассмотрим базовые подходы и методы магии под кодовым названием Feature Engineering.

Цели данного модуля:

Научиться создавать новые признаки в данных с помощью базовых операций со столбцами.
Освоить методы работы с датой и временем в Pandas.
Научиться применять собственные функции для преобразования столбцов и создания новых признаков.
Познакомиться с новым типом данных category и научиться использовать его при работе с данными.

## ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

В этом модуле мы продолжим работу с датасетом о продажах объектов недвижимости в Мельбурне и его пригородах. Напомним, что в прошлый раз мы совершили несколько преобразований типов столбцов в нашей изначальной таблице, когда исследовали её структуру.

Скачать обновлённый набор данных в формате csv (разделитель — ',') можно https://lms.skillfactory.ru/assets/courseware/v1/5ede138b70107a618949677b325096ca/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/melb_data_ps.zip (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Чтобы освежить в памяти информацию о столбцах таблицы, вы можете вернуться сюда.

Импортируем Pandas, прочитаем наш csv-файл в DataFrame и выведем первые пять строк таблицы, чтобы убедиться в том, что файл прочитан верно.

In [1]:
import pandas as pd

In [2]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


# 2. Базовые операции со столбцами DataFrame

## СОЗДАНИЕ КОПИИ ТАБЛИЦЫ

На протяжении всего модуля мы будем производить множество тренировочных преобразований с нашей таблицей. Поэтому, чтобы не переопределять переменную melb_data и тем самым не повредить первоначальный DataFrame, создадим копию melb_df с помощью метода copy():

In [3]:
melb_df = melb_data.copy()
melb_df.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


⭐ Лайфхак. Старайтесь всегда оставлять переменную с первоначальным DataFrame неизменной, создавайте копию исходной таблицы и совершайте преобразования на ней. Это оградит вас от ошибок, которые можно совершить при подготовке данных. Например, если вы понимаете, что преобразование оказалось неудачным, достаточно будет лишь запустить ячейку, в которой вы производите копирование, а не читать таблицу заново. Особенно критичным это может быть, когда количество строк в таблице исчисляется миллионами и её чтение занимает до нескольких минут.

## УДАЛЕНИЕ СТОЛБЦОВ

Среди списка базовых операций над столбцами в Pandas важное место занимает возможность удаления столбцов из таблицы. Это может быть полезно, например, когда в данных есть признаки, которые не несут полезной информации.

Представим, что мы хотим построить модель, которая бы предсказывала цену объекта недвижимости в Мельбурне. Даже не будучи профессиональными риелторами, мы можем легко сделать следующие выводы:

цена объекта никак не зависит от его порядкового номера (столбец index);
признак, описывающий долготу и широту в виде кортежа Coordinates, дублирует информацию, представленную в столбцах Longitude и Lattitude.
За удаление строк и столбцов в таблице отвечает метод drop().

Основные параметры метода drop()

labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;

axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;

inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает;
если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; 
по умолчанию параметр равен False.

Удалим столбцы index и Coordinates из таблицы с помощью метода drop(). Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.

In [4]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()

#melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
#melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


## МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

→ Pandas поддерживает базовые математические операции между столбцами: столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Причём все операции со столбцами совершаются поэлементно, очень быстро, а самое главное — без написания циклов.

Такая производительность достигается за счёт того, что все математические операции со столбцами выполняются на языке программирования С, что значительно повышает скорость вычислений по сравнению с перебором элементов в цикле. 

Например, давайте создадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

In [5]:
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

А теперь введём признак MeanRoomsSquare, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:

In [6]:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

Можно ввести ещё один интересный признак — AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму:

In [7]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area / sum_area
display(melb_df['AreaRatio'])

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.

Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.

Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

In [8]:
customer_df = pd.DataFrame({
        'number': [0, 1, 2, 3, 4],
        'cust_id': [128, 1201, 9832, 4392, 7472],
        'cust_age': [13, 21, 19, 21, 60],
        'cust_sale': [0, 0, 0.2, 0.15, 0.3],
        'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
        'cust_order': [1400, 14142, 900, 1240, 8430]
    })

def delete_columns(df, col=[]):
    try:
        new_df = df.drop(col, axis=1)
        return new_df
    except KeyError:
        return None

print(delete_columns(df = customer_df, col = ['cust_sale2', 'not_existing_col']))


None


In [9]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

countries_df['Density'] = (countries_df['population'] * 1e6) / countries_df['square']
round((countries_df['Density'].mean()), 2)

84.93

# 3. Работа с датами в DataFrame

## ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ

→ При генерации новых признаков очень ценным может стать временной признак (признак даты и времени). Это особый тип данных, с которым приходится сталкиваться в большинстве задач по обработке данных. В реальных задачах часто нужно сравнивать даты, выделять день недели или час, вычислять различные интервалы между датами. 

Например, в наших данных об объектах недвижимости есть признак даты продажи — столбец Date. Из данного признака можно выделить массу полезной информации, например год, месяц или день недели продажи имущества. На рынке недвижимости, как известно, присутствует сезонность: есть периоды, когда недвижимость покупается чаще, а есть интервалы времени, когда рынок претерпевает застой, поэтому было бы неплохо учитывать эту сезонность при анализе рынка.

## ФОРМАТ DATETIME

В жизни мы видим даты в привычных для нас форматах. Например, запись 01.12.18 обычно означает 1 декабря 2018 года. Однако для жителей США эта дата окажется 12 января 2018 года, так как для них привычнее сначала указывать номер месяца, а затем день. 

Многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:

2018-11-09 15:45:21;
11/09/2018 3:45:20 PM;
2018-11-09T15:45:21.2984.
Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой. Для этого был создан единый способ обозначения даты и времени. 

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [10]:
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [11]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64. Рассмотрим несколько возможностей этого типа данных.

## ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

date — дата;

year, month, day — год, месяц, день;

time — время;

hour, minute, second — час, минута, секунда;

dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;

day_name — название дня недели;

dayofyear — порядковый день года;

quarter — квартал (интервал в три месяца).

Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью pd.to_datetime().

Обратите внимание, что вы не сможете обратиться к аксессору, если ваш столбец не приведён к типу datetime.

С использованием dt можно подробнее ознакомиться [здесь](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [12]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:',years_sold.min())
print('Max year sold:',years_sold.max())
print('Mode year sold:',years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.

Из результатов видно, что данные представлены за интервал с 2016 по 2017 год и наибольшее количество объектов было продано в 2017 году.

Примечание. Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [13]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно).

## РАБОТА С ИНТЕРВАЛАМИ

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [14]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [15]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [16]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [17]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [18]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

In [19]:
weekend_count = sum(melb_df['WeekdaySale'] == 5) + sum(melb_df['WeekdaySale'] == 6)

#weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]


display(weekend_count)

12822

Вам представлены данные (в формате csv) об отчётах очевидцев НЛО в США за период с 1930 по 2020 год.

В данных есть следующие признаки:

"City" — город, где был замечен НЛО;

"Colors Reported" — цвет объекта;

"Shape Reported" — форма объекта;

"State" — обозначение штата;

"Time" — время, когда был замечен НЛО (данные отсортированы от старых наблюдений к новым). 

Прочитайте данные, сделайте преобразование времени к формату datetime и выполните задания ниже.

In [20]:
ufo_df = pd.read_csv('data/ufo.txt', sep=',')
display(ufo_df.head())

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


In [21]:
ufo_df['Time'] = pd.to_datetime(ufo_df['Time'],dayfirst=False)
display(ufo_df['Time'])

0       1930-06-01 22:00:00
1       1930-06-30 20:00:00
2       1931-02-15 14:00:00
3       1931-06-01 13:00:00
4       1933-04-18 19:00:00
                ...        
18236   2000-12-31 23:00:00
18237   2000-12-31 23:00:00
18238   2000-12-31 23:45:00
18239   2000-12-31 23:45:00
18240   2000-12-31 23:59:00
Name: Time, Length: 18241, dtype: datetime64[ns]

In [22]:
ufo_df['Time'].dt.year.mode()

0    1999
Name: Time, dtype: int64

In [23]:
ufo_df['Date'] = ufo_df['Time'].dt.date
print(ufo_df[ufo_df['State'] == 'NV']['Date'].diff().dt.days.mean())

68.92932862190813


# 4. Создание и преобразование столбцов с помощью функций

 Библиотека Pandas предоставляет большое количество возможностей для преобразований данных, однако иногда необходимо совершать более сложные манипуляции над столбцами. Например, из столбцов, содержащих в себе некоторый текст, необходимо специальным образом извлечь определённые слова, даты или числа.

→ Для таких случаев Pandas не имеет специальных методов, однако позволяет расширить свою функциональность за счёт использования пользовательских функций.

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода nunique():

In [24]:
melb_df['Address'].nunique()

13378

Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет **статистической значимости**, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца *Address*:

In [25]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [26]:
# Создаём список географических пометок exclude_list.
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
# Возвращаем переменную street_type, в которой хранится подтип улицы.


def get_street_type(address):
    exculde_list = ['N', 'S', 'W', 'E']
    adress_list = address.split(' ')
    street_type = adress_list[-1]
    if street_type in exculde_list:
        street_type = adress_list[-2]
    return street_type


ДОПОЛНИТЕЛЬНО

Подробнее о методе [split()](https://docs-python.ru/tutorial/operatsii-tekstovymi-strokami-str-python/metod-str-split/)

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию *get_street_type* в аргумент метода столбца *apply()*. В результате получим объект *Series*, который положим в переменную *street_types*:

In [27]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Обратите внимание, что функция пишется для одного элемента столбца, а метод **apply()** применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

In [28]:
street_types.nunique()

56

У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода **value_counts**:

In [29]:
street_types.value_counts()

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим *n* подтипов, которые встречаются чаще всего, а остальные обозначим как *'other'* (другие).

Для этого к результату метода *value_counts* применим метод *nlargest()*, который возвращает n наибольших значений из *Series*. Зададим *n=10*, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута *index*, а результат занесём в переменную *popular_stypes*:

In [30]:
types = {'Avenue':'Av','Boulevard':'Bv','Parade':'Pde'}
popular_stypes = street_types.apply(lambda x: x if x not in types else types[x])
popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


Попробуйте применить использованные методы последовательно и выводить результат, чтобы проследить за логикой преобразования.

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [31]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

Посмотрим на результирующее число уникальных подтипов:

In [32]:
print(melb_df['StreetType'].nunique())

11


Теперь, у нас нет потребности хранить признак *Address*, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [33]:
melb_df = melb_df.drop('Address', axis=1)

Таким образом, с помощью написания собственных функций и их комбинирования с методом apply() из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы. 

Примечание. Внимательный читатель наверняка обратит внимание на то, что мы допустили небольшую ошибку!

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

Мы предлагаем вам самостоятельно разобраться с этой ошибкой: попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.

→ Обратите внимание, что данное преобразование необходимо применить до сокращения количества уникальных категорий.

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1. Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2. Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.

3. Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4. Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

5. Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».



→ Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.

In [34]:
def get_weekend(weekday):
    if weekday in [5,6]:
        return 1
    return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean()))

1081199

In [35]:
top_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] =  melb_df['SellerG'].apply(lambda x: x if x in top_sellers else 'other')

In [36]:
melb_df['SellerG'].value_counts()

Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Eview              51
Purplebricks       51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Nick               40
Alexkarbon

In [37]:
print(round(melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min(), 1))

1.3


In [38]:
def get_experience(arg):
    year_vars = ['год','года','лет']
    month_vars = ['месяц','месяца','месяцев']
    check = arg.split(' ')
    exp = 0
    for word in check:
        if word in year_vars:
            exp += int(check[check.index(word) - 1]) * 12
        if word in month_vars:
            exp += int(check[check.index(word) - 1])
    return exp

def get_experience_2(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)

# 5. Тип данных Category

## ПРИЗНАКИ: КАТЕГОРИАЛЬНЫЕ И ЧИСЛОВЫЕ

Рассмотрим такие статистические термины, как категориальные и числовые признаки.

Под числовыми признаками обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.

Числовые признаки могут быть:
* дискретными (например, количество комнат, пациентов, дней);
* непрерывными (например, масса, цена, площадь).
* Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Под категориальными признаками обычно подразумевают столбцы в таблице, которые обозначают принадлежность объекта к какому-то классу/категории.

Категориальные признаки могут быть:

* номинальными (например, пол, национальность, район);
* порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).
Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.

Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.

Возникает вопрос: а какие признаки стоит считать категориальными?

Однозначного ответа на этот вопрос нет. Решение, какой признак отнести к классу категорий, остаётся за исследователем. Некоторые специалисты даже относят количественные признаки в разряд категориальных, когда диапазон возможных значений довольно мал. 

Не существует жёсткого правила, определяющего, сколько значений должна иметь категориальная переменная. Вы должны использовать собственные знания о предметной области, чтобы сделать выбор. Однако в этом разделе мы всё же введём некоторые рекомендации.

ДОПОЛНИТЕЛЬНО

На самом деле классификация признаков уходит даже глубже — [здесь](https://medium.com/nuances-of-programming/статистические-типы-данных-используемые-в-машинном-обучении-b8891039b09) вы сможете подробнее ознакомиться со всеми тонкостями.

### ПОЧЕМУ ТАК ВАЖНО ОТЛИЧАТЬ КАТЕГОРИАЛЬНЫЕ СТОЛБЦЫ ОТ ДРУГИХ?

→ Оказывается, анализ и предобработка категориальных признаков отличается от предобработки числовых признаков.

Мы уже видели, что в столбцах с типом данных object мы не можем рассчитать среднее значение, стандартное отклонение или другие статистические параметры, которые годятся только для чисел.

Способ заполнения пропущенных данных и поиск аномальных значений также различаются для разных типов признаков, но этот момент мы рассмотрим в модуле по очистке данных.

## КАТЕГОРИИ В ДАННЫХ О НЕДВИЖИМОСТИ

Давайте определим число уникальных категорий в каждом столбце нашей таблицы *melb_df*. Для этого создадим вспомогательную таблицу *unique_counts*:

In [39]:
# создаём пустой список
# пробегаемся по именам столбцов в таблице
# создаём кортеж (имя столбца, число уникальных значений)
# добавляем кортеж в список
# создаём вспомогательную таблицу и сортируем её
# выводим её на экран

unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes)
    unique_list.append(item)

unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique', ignore_index=True)

display(unique_counts)


,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,5,int64
4,Regionname,8,object
5,Bathroom,9,int64
6,Rooms,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


Разберём код подробнее:

1. Создаём пустой список, в который будем добавлять кортежи: имя столбца, количество уникальных значений в нём и тип столбца.
2. В цикле перебираем имена столбцов, которые получаем с помощью атрибута columns. В переменной col на каждой итерации находятся имена столбцов — обращаемся к ним в цикле и извлекаем число уникальных элементов с помощью метода nunique(), а также тип столбца с помощью атрибута dtypes. Результат заносим в кортеж и добавляем его в список.
3. Из списка с кортежами (имя столбца, количество уникальных значений в нём, тип столбца) создаём DataFrame, даём названия его столбцам: Column_Name, Num_unique и Type.
4. Сортируем таблицу по столбцу Num_unique в порядке возрастания количества уникальных элементов с помощью метода sort_values() и выводим результат на экран.

Примечание. Мы ещё не изучали сортировку DataFrame методом sort_values() — данную тему мы обсудим в следующем модуле, однако здесь эта функция необходима для более наглядной интерпретации результата.

Итак, с помощью такого несложного кода мы смогли получить вспомогательную таблицу, где в отсортированном виде показано количество уникальных значений во всех столбцах и их типы.

?
Что интересного мы можем узнать из такой таблицы?

Если присмотреться внимательно, можно увидеть резкий скачок количества уникальных значений, начиная с 14 строки таблицы, где число уникальных значений составляет 152. Учтём этот момент.

Условимся, что категориальными будем считать признаки, у которых число уникальных категорий меньше 150. 

Однако учтём, что признак Date (дата продажи), преобразованный нами ранее в формат datetime, является временным признаком, поэтому далее не будем его воспринимать как категориальный. 

К тому же в наш потенциальный список попали количественные столбцы Rooms, Car, Bedroom и Bathroom. Договоримся, что мы не будем относить их к разряду категориальных, однако, как упоминалось ранее, такое тоже вполне возможно.

Примечание. Ещё раз подчеркиваем, что такая классификация признаков является исключительно субъективной и специфична для задачи.

## ТИП ДАННЫХ CATEGORY

Для хранения и оптимизации работы с категориальными признаками в Pandas предусмотрен специальный тип данных — category.

→ Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. Так как данные вместо изначальных строк хранятся в памяти как число, то объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается, что повышает эффективность хранения и работы с таблицей.

→ Более того, этот тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным (что сложно сделать для типа данных object). Также резко повышается производительность операций, совершаемых с такими столбцами.

Самый простой способ преобразования столбцов к типу данных category — это использование уже знакомого нам метода **astype()**, в параметры которого достаточно передать строку **'category'**.

Рассмотрим это на примере.

Для начала, выведем информацию о памяти, занимаемой текущей таблицей, с помощью метода **info()**:

In [40]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  object        
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  object        
 5   SellerG          13580 non-null  object        
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  objec

None

Текущий объём памяти — 2.7 Мб.

Примечание. Объём занимаемой памяти и количество столбцов на вашем компьютере может отличаться в зависимости от того, выполняли ли вы код и задания из предыдущих разделов на добавление и преобразование данных в таблице.

Сделаем преобразование столбцов к типу данных category:

In [41]:
cols_to_exclude = ['Date', 'Rooms', 'Car', 'Bedroom', 'Bathroom']
max_unique_count = 150
for col in melb_df.columns:
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')

display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

Разберём код подробнее:
1. Задаём список столбцов, которые мы не берём в рассмотрение (cols_to_exclude), а также условленный нами ранее порог уникальных значений столбца max_unique_count.
2. В цикле перебираем имена столбцов, и, если число уникальных категорий меньше заданного порога и имён столбцов нет в списке cols_to_exclude, то с помощью метода astype() приводим столбец к типу данных category.
3. Итоговый объём памяти — 1.9 Мб. В результате такого преобразования объём памяти, занимаемый таблицей, уменьшился почти в 1.5 раза. Это впечатляет!

Особенно хорошо такое преобразование работает на действительно больших данных, где число строк превышает сотни тысяч или миллионы. Иногда изменение типов может уменьшить объём памяти в десятки раз и существенно увеличить производительность.

### ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY

У типа данных category есть свой специальный аксесcор **cat**, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце **Regionname**:

In [42]:
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


А теперь посмотрим, каким образом столбец кодируется в виде чисел в памяти компьютера. Для этого можно воспользоваться атрибутом **codes**:

In [43]:
display(melb_df['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.

Рассмотрим на примере: переименуем категории признака типа постройки *Type* — заменим их на полные названия (напомним, **u — unit**, **h — house**, **t — townhouse**).

In [44]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

ДОПОЛНИТЕЛЬНО

Подробнее ознакомиться с другими возможностями типа данных category можно [здесь](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html).

### ПОДВОДНЫЕ КАМНИ

А теперь представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры **(flat)**. Создадим объект **Series new_houses_types**, в котором будем хранить типы зданий новой партии домов. Преобразуем тип **new_houses_types** в такой же тип, как и у столбца Type в таблице **melb_data**, и выведем результат на экран:

In [45]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Хммм... С нашими новыми объектами недвижимости произошло нечто странное. По какой-то причине вместо квартир мы получили пустые значения — NaN.

На самом деле причина проста: тип данных category хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.

Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

In [46]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

Примечание. Добавление новой категории в столбец Type не отразится на самом столбце — текущие категории не изменятся, однако такое преобразование позволит добавлять в таблицу новые данные о домах с новой категорией — flat.

Из данного примера можно сделать вывод, что если набор категорий в столбце жёстко не зафиксирован и может обновляться в процессе работы, то тип category не является подходящим типом данных для этого столбца или необходимо постоянно писать проверки при обновлении таблицы.

Теперь, когда мы рассмотрели основные моменты и нюансы работы с типом данных category, можно сформулировать несколько рекомендаций по его использованию:
1. Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.
2. Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.
3. Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.
4. Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.

In [47]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [48]:
top_suburb_list = melb_df['Suburb'].value_counts()
top_suburb_list = pd.Series(top_suburb_list[:119])
top_suburb_list = top_suburb_list.index
display(top_suburb_list)

for item in range(len(melb_df['Suburb'])):
    if melb_df['Suburb'].loc[item] not in top_suburb_list:
        melb_df['Suburb'].loc[item] = 'other'


Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', length=119)

C:\Users\22775\AppData\Local\Temp\ipykernel_37628\3404157333.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melb_df['Suburb'].loc[item] = 'other'


In [49]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')

In [50]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

# 6. Закрепление знаний

На этот раз мы поговорим о спорте, а точнее — о велоспорте. 

Вашим заданием в этом модуле будет проанализировать и преобразовать данные о велопоездках клиентов компании Citi Bike (США), специализирующейся на прокате велосипедов.

Скачать датасет в формате csv можно здесь.

Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

* starttime — время начала поездки (дата, время);
* stoptime — время окончания поездки (дата, время);
* start station id — идентификатор стартовой стоянки;
* start station name — название стартовой стоянки;
* start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
* end station id — идентификатор конечной стоянки;
* end station name — название конечной стоянки;
* end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
* bikeid — идентификатор велосипеда;
* usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
* birth year — год рождения клиента;
* gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).
Оригинальный источник данных вы можете найти здесь. Он представляет собой таблицу на 1,7 миллиона строк. В учебных целях мы удалили из датасета несколько признаков, а также уменьшили число строк.

In [51]:
citi_bike = pd.read_csv('data/citibike-tripdata.csv', sep=',')
display(citi_bike.head())

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [52]:
citi_bike['start station id'].isna().sum()

169

In [53]:
citi_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

In [54]:
display(citi_bike['start station id'].mode())
display(citi_bike['bikeid'].mode())

0    281.0
Name: start station id, dtype: float64

0    33887
Name: bikeid, dtype: int64

In [55]:
display(round(citi_bike['usertype'].value_counts(normalize=True), 2))
display(citi_bike['gender'].value_counts())

Subscriber    0.77
Customer      0.23
Name: usertype, dtype: float64

1    183582
2     74506
0     41912
Name: gender, dtype: int64

In [56]:
display(citi_bike['start station id'].nunique() != citi_bike['end station id'].nunique())
citi_bike['starttime'] = pd.to_datetime(citi_bike['starttime'])
citi_bike['Age'] = citi_bike['starttime'].dt.year - citi_bike['birth year']
display(citi_bike['Age'].min() == 10)
display(citi_bike['start station name'].value_counts(ascending=True))
display(citi_bike['end station name'].mode())

True

False

Eastern Pkwy & Washington Ave           6
47 Ave & Skillman Ave                   7
Railroad Ave & Kay Ave                  9
Franklin Ave & Empire Blvd             11
NYCBS Depot - GOW                      15
                                     ... 
West St & Chambers St                1835
12 Ave & W 40 St                     1845
Pershing Square North                1873
Central Park S & 6 Ave               1909
Grand Army Plaza & Central Park S    1928
Name: start station name, Length: 759, dtype: int64

0    West St & Chambers St
Name: end station name, dtype: object

In [62]:
citi_bike = citi_bike.drop(['start station id','end station id'], axis=1)
display(citi_bike[citi_bike['Age'] > 60].shape[0])

11837

In [63]:
citi_bike = citi_bike.drop('birth year', axis=1)

KeyError: "['birth year'] not found in axis"

In [64]:
citi_bike['stoptime'] = pd.to_datetime(citi_bike['stoptime'])

In [65]:
citi_bike['trip duration'] = (citi_bike['stoptime'] - citi_bike['starttime'])

In [74]:
citi_bike['weekend'] = citi_bike['starttime'].dt.day_of_week.apply(lambda x: 1 if x in [5,6] else 0)
display(citi_bike['weekend'])

0         1
1         1
2         1
3         1
4         1
         ..
299995    0
299996    0
299997    0
299998    0
299999    0
Name: weekend, Length: 300000, dtype: int64

In [76]:
citi_bike[citi_bike['weekend'] == 1].shape[0]

115135

In [79]:
def daytime(time):
    if time in range(0,7):
        return 'night'
    if time in range(7,13):
        return 'morning'
    if time in range(13,19):
        return 'day'
    if time in range(19,24):
        return 'evening'

citi_bike['time_of_day'] = citi_bike['starttime'].dt.hour.apply(daytime)

In [81]:
citi_bike[citi_bike['time_of_day'] == 'day'].shape[0] / citi_bike[citi_bike['time_of_day'] == 'night'].shape[0]


9.480411004308916